In [1]:
import numpy as np
import matplotlib.pyplot as plt
import dedalus.public as d3
import logging
logger = logging.getLogger(__name__)

hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).


In [14]:
Lx, Lz = 1, 2
Nx, Nz = 128, 256
Reynolds = 5e4
Schmidt = 1
modes = 4
stop_sim_time = 10
timestepper = d3.RK222
max_timestep = 5e-2
dtype = np.float64

In [17]:
coords = d3.CartesianCoordinates('x', 'z')
dist = d3.Distributor(coords, dtype=dtype)
xbasis = d3.RealFourier(coords['x'], size=Nx, bounds=(0, Lx))
zbasis = d3.RealFourier(coords['z'], size=Nz, bounds=(-Lz/2, Lz/2))

# Fields
p = dist.Field(name='p', bases=(xbasis,zbasis))
s = dist.Field(name='s', bases=(xbasis,zbasis))
u = dist.VectorField(coords, name='u', bases=(xbasis,zbasis))
tau_p = dist.Field(name='tau_p')


In [20]:
# Substitutions
nu = 1 / Reynolds
D = nu / Schmidt
x, z = dist.local_grids(xbasis, zbasis)
ex, ez = coords.unit_vector_fields(dist)

# Problem
problem = d3.IVP([u, s, p, tau_p], namespace=locals())
problem.add_equation("dt(u) + grad(p) - nu*lap(u) = - u@grad(u)")
problem.add_equation("dt(s) - D*lap(s) = - u@grad(s)")
problem.add_equation("div(u) + tau_p = 0")
problem.add_equation("integ(p) = 0") # Pressure gauge


{'LHS': Integrate(Integrate(<Field 139674192980624>)),
 'RHS': 0,
 'condition': 'True',
 'tensorsig': (),
 'dtype': numpy.float64,
 'M': 0,
 'L': Integrate(Integrate(<Field 139674192980624>)),
 'F': <Field 139674419371280>,
 'domain': <dedalus.core.domain.Domain at 0x7f086eb2c890>,
 'matrix_dependence': array([ True,  True]),
 'matrix_coupling': array([False, False])}

In [22]:
# Solver
solver = problem.build_solver(timestepper)
solver.stop_sim_time = stop_sim_time

# Initial conditions
# Background shear
u['g'][0] = 1/2 + 1/2 * (np.tanh((z-0.5)/0.1) - np.tanh((z+0.5)/0.1))
# Match tracer to shear
s['g'] = u['g'][0]
# Add small vertical velocity perturbations localized to the shear layers
u['g'][1] += 0.1 * np.sin(2*np.pi*x/Lx) * np.exp(-(z-0.5)**2/0.01)
u['g'][1] += 0.1 * np.sin(2*np.pi*x/Lx) * np.exp(-(z+0.5)**2/0.01)

# Analysis
snapshots = solver.evaluator.add_file_handler('snapshots', sim_dt=0.1, max_writes=10)
snapshots.add_task(s, name='tracer')
snapshots.add_task(p, name='pressure')
snapshots.add_task(-d3.div(d3.skew(u)), name='vorticity')

# CFL
CFL = d3.CFL(solver, initial_dt=max_timestep, cadence=10, safety=0.2, threshold=0.1,
             max_change=1.5, min_change=0.5, max_dt=max_timestep)
CFL.add_velocity(u)

# Flow properties
flow = d3.GlobalFlowProperty(solver, cadence=10)
flow.add_property((u@ez)**2, name='w2')

2023-05-23 22:32:07,064 subsystems 0/1 INFO :: Building subproblem matrices 1/64 (~2%) Elapsed: 0s, Remaining: 3s, Rate: 2.4e+01/s
2023-05-23 22:32:07,225 subsystems 0/1 INFO :: Building subproblem matrices 7/64 (~11%) Elapsed: 0s, Remaining: 2s, Rate: 3.4e+01/s
2023-05-23 22:32:07,381 subsystems 0/1 INFO :: Building subproblem matrices 14/64 (~22%) Elapsed: 0s, Remaining: 1s, Rate: 3.9e+01/s
2023-05-23 22:32:07,551 subsystems 0/1 INFO :: Building subproblem matrices 21/64 (~33%) Elapsed: 1s, Remaining: 1s, Rate: 4.0e+01/s
2023-05-23 22:32:07,764 subsystems 0/1 INFO :: Building subproblem matrices 28/64 (~44%) Elapsed: 1s, Remaining: 1s, Rate: 3.8e+01/s
2023-05-23 22:32:08,006 subsystems 0/1 INFO :: Building subproblem matrices 35/64 (~55%) Elapsed: 1s, Remaining: 1s, Rate: 3.6e+01/s
2023-05-23 22:32:08,183 subsystems 0/1 INFO :: Building subproblem matrices 42/64 (~66%) Elapsed: 1s, Remaining: 1s, Rate: 3.6e+01/s
2023-05-23 22:32:08,346 subsystems 0/1 INFO :: Building subproblem matri

In [23]:
# Main loop
try:
    logger.info('Starting main loop')
    while solver.proceed:
        timestep = CFL.compute_timestep()
        solver.step(timestep)
        if (solver.iteration-1) % 10 == 0:
            max_w = np.sqrt(flow.max('w2'))
            logger.info('Iteration=%i, Time=%e, dt=%e, max(w)=%f' %(solver.iteration, solver.sim_time, timestep, max_w))
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    solver.log_stats()

2023-05-23 22:32:21,548 __main__ 0/1 INFO :: Starting main loop
2023-05-23 22:32:21,881 __main__ 0/1 INFO :: Iteration=1, Time=5.000000e-02, dt=5.000000e-02, max(w)=0.100000
2023-05-23 22:32:22,234 __main__ 0/1 INFO :: Iteration=11, Time=5.500000e-01, dt=5.000000e-02, max(w)=0.043030
2023-05-23 22:32:22,738 __main__ 0/1 INFO :: Iteration=21, Time=8.000000e-01, dt=2.500000e-02, max(w)=nan
2023-05-23 22:32:23,171 __main__ 0/1 INFO :: Iteration=31, Time=1.050000e+00, dt=2.500000e-02, max(w)=nan
2023-05-23 22:32:23,603 __main__ 0/1 INFO :: Iteration=41, Time=1.300000e+00, dt=2.500000e-02, max(w)=nan
2023-05-23 22:32:23,975 __main__ 0/1 INFO :: Iteration=51, Time=1.550000e+00, dt=2.500000e-02, max(w)=nan
2023-05-23 22:32:24,324 __main__ 0/1 INFO :: Iteration=61, Time=1.800000e+00, dt=2.500000e-02, max(w)=nan
2023-05-23 22:32:24,684 __main__ 0/1 INFO :: Iteration=71, Time=2.050000e+00, dt=2.500000e-02, max(w)=nan
2023-05-23 22:32:25,027 __main__ 0/1 INFO :: Iteration=81, Time=2.300000e+00, d